In [1]:
# import pip

# pip.main(['install', 'mmh3'])

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
25,application_1618594989241_0026,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
from pyspark import SparkContext
from datetime import datetime
import random
import math

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
sc = spark.sparkContext
sc.install_pypi_package("mmh3")
import mmh3


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Using cached https://files.pythonhosted.org/packages/25/2b/a402183507dea33ecce49caca52839c6d00552c0f556853af13884fee605/mmh3-3.0.0-cp37-cp37m-manylinux1_x86_64.whl

In [5]:
metadata = sc.textFile("s3://diomdle2021/movie.metadata.tsv")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
k_gram_size=9

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
 metadata = metadata.map(lambda line: line.split("\t")).map(lambda a: (int(a[0]),a[2]))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
metadata.take(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[(975900, 'Ghosts of Mars'), (3196793, 'Getting Away with Murder: The JonBenét Ramsey Mystery'), (28463795, 'Brun bitter'), (9363483, 'White Of The Eye'), (261236, 'A Woman in Flames'), (13696889, 'The Gangsters'), (18998739, "The Sorcerer's Apprentice"), (10408933, "Alexander's Ragtime Band"), (9997961, 'Contigo y aquí'), (2345652, 'City of the Dead')]

In [9]:
movie_plots = sc.textFile("s3://diomdle2021/plot_summaries.txt")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
movie_plots = movie_plots.map(lambda line: line.split("\t")).map(lambda a: (int(a[0]),a[1]))
movie_plots = movie_plots.sample(withReplacement=False, fraction=0.25)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
movie_plots.take(1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[(595909, 'Seventh-day Adventist Church pastor Michael Chamberlain, his wife Lindy, their two sons, and their nine-week-old daughter Azaria are on a camping holiday in the Outback. With the baby sleeping in their tent, the family is enjoying a barbecue with their fellow campers when a cry is heard. Lindy returns to the tent to check on Azaria and is certain she sees a dingo with something in its mouth running off as she approaches. When she discovers the infant is missing, everyone joins forces to search for her, without success. It is assumed what Lindy saw was the animal carrying off the child, and a subsequent inquest rules her account of events is true. The tide of public opinion soon turns against the Chamberlains. For many, Lindy seems too stoic, too cold-hearted, and too accepting of the disaster that has befallen her. Gossip about her begins to swell and soon is accepted as statements of fact. The couple\'s beliefs are not widely practised in the country, and when the media rep

In [16]:
def break_k_grams(plot):
    shingles = []
    for i in range(len(plot)-k_gram_size+1):
        shingles.append(plot[i:i+k_gram_size])
        
    return shingles

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
movie_shingles = movie_plots.mapValues(break_k_grams)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
movie_shingles.take(1)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[(595909, ['Seventh-d', 'eventh-da', 'venth-day', 'enth-day ', 'nth-day A', 'th-day Ad', 'h-day Adv', '-day Adve', 'day Adven', 'ay Advent', 'y Adventi', ' Adventis', 'Adventist', 'dventist ', 'ventist C', 'entist Ch', 'ntist Chu', 'tist Chur', 'ist Churc', 'st Church', 't Church ', ' Church p', 'Church pa', 'hurch pas', 'urch past', 'rch pasto', 'ch pastor', 'h pastor ', ' pastor M', 'pastor Mi', 'astor Mic', 'stor Mich', 'tor Micha', 'or Michae', 'r Michael', ' Michael ', 'Michael C', 'ichael Ch', 'chael Cha', 'hael Cham', 'ael Chamb', 'el Chambe', 'l Chamber', ' Chamberl', 'Chamberla', 'hamberlai', 'amberlain', 'mberlain,', 'berlain, ', 'erlain, h', 'rlain, hi', 'lain, his', 'ain, his ', 'in, his w', 'n, his wi', ', his wif', ' his wife', 'his wife ', 'is wife L', 's wife Li', ' wife Lin', 'wife Lind', 'ife Lindy', 'fe Lindy,', 'e Lindy, ', ' Lindy, t', 'Lindy, th', 'indy, the', 'ndy, thei', 'dy, their', 'y, their ', ', their t', ' their tw', 'their two', 'heir two ', 'eir two s', '

In [29]:
#optional but advised hashing of shingles into 4 bytes (%2**4*8)
movie_shingles = movie_shingles.mapValues(lambda shing: sorted([mmh3.hash(x.lower(),31)%(2**32) for x in shing]))
#movie_shingles = movie_shingles.mapValues(lambda shing: sorted([x for x in shing]))

movie_shingles.take(2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[(595909, [1610381, 2963522, 3536910, 7559499, 7886662, 8373689, 8821477, 10218147, 10915134, 21712442, 23441207, 23441207, 23441207, 25062129, 25398113, 25831237, 26640185, 28906870, 31743004, 32290540, 35522016, 37150410, 38832472, 39395145, 39718398, 41425136, 43724211, 44177293, 46442968, 47314876, 49333193, 54924571, 56544702, 63721437, 64293408, 64864531, 65946554, 68339776, 73081184, 74760099, 75586681, 75860616, 75904518, 77453027, 78778116, 79044156, 80555416, 82175385, 82645097, 82645097, 86035008, 87872340, 89874163, 92004528, 93995139, 94292776, 94875194, 99914756, 100597461, 103026509, 106125609, 108612517, 109350610, 110864760, 112843591, 120966122, 122634625, 123410125, 128062083, 128949401, 129301358, 135401296, 137952751, 146235470, 146267853, 146382614, 149369257, 149406614, 151743082, 156822706, 156942617, 158215518, 160301258, 160747259, 161834125, 162889582, 163659722, 164070859, 164486573, 165522735, 167963727, 168102981, 169484113, 169718088, 173442562, 176924592

In [30]:
def random_hash():
    a = random.randint(1,1000)
    b = random.randint(0,1000)
    return a,b

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
random_hash()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(138, 555)

In [32]:
#n_shingles = 23254807 
n_shingles = movie_shingles.flatMap(lambda tup: tup[1]).distinct().count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [36]:
#prime number for hashing
p = 24000001
#number of hash functions
k=100
hash_functions = []
for i in range(k):
    hash_functions.append(random_hash())
hash_functions

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[(576, 890), (452, 849), (56, 716), (870, 525), (202, 503), (416, 756), (112, 832), (386, 486), (817, 255), (299, 817), (662, 813), (699, 153), (498, 505), (187, 524), (441, 515), (274, 467), (359, 105), (71, 266), (602, 991), (295, 193), (499, 683), (998, 6), (518, 116), (237, 547), (949, 688), (955, 103), (365, 423), (367, 976), (398, 918), (134, 339), (724, 125), (854, 476), (88, 808), (104, 644), (890, 159), (865, 409), (467, 614), (153, 827), (979, 162), (624, 88), (791, 572), (681, 365), (829, 933), (835, 474), (807, 150), (727, 800), (702, 267), (692, 735), (305, 82), (213, 17), (324, 976), (372, 976), (8, 936), (730, 818), (337, 0), (698, 932), (73, 194), (631, 266), (13, 826), (553, 278), (150, 27), (495, 605), (581, 898), (25, 922), (877, 626), (883, 137), (395, 194), (255, 115), (757, 541), (148, 685), (912, 539), (170, 196), (544, 684), (803, 946), (787, 59), (310, 225), (588, 435), (992, 917), (443, 846), (663, 62), (642, 307), (180, 316), (242, 142), (208, 433), (548, 992

In [37]:
def get_signature(hashed_shingles):
    signature_matrix = {}
    for element in hashed_shingles:
        for i in range(k):
            hash_result = calc_hash(*hash_functions[i],p,n_shingles, element)
            if  hash_result < signature_matrix.get(i,float('inf')):
                signature_matrix[i]=hash_result
    return signature_matrix

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [38]:
def calc_hash(a,b,prime,n,value):
    return ((a*value + b)%prime)%n
    

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [39]:
movie_signatures = movie_shingles.mapValues(get_signature)
movie_signatures.take(2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[(595909, {0: 101, 1: 774, 2: 4010, 3: 2254, 4: 2961, 5: 939, 6: 2471, 7: 7471, 8: 1703, 9: 7642, 10: 1166, 11: 638, 12: 1177, 13: 2056, 14: 3760, 15: 1923, 16: 1679, 17: 759, 18: 657, 19: 5, 20: 98, 21: 2692, 22: 2609, 23: 1081, 24: 2019, 25: 486, 26: 623, 27: 1502, 28: 909, 29: 1035, 30: 2665, 31: 1172, 32: 369, 33: 5234, 34: 2379, 35: 601, 36: 1386, 37: 1087, 38: 2604, 39: 2493, 40: 872, 41: 109, 42: 520, 43: 2323, 44: 676, 45: 1370, 46: 9135, 47: 3204, 48: 11646, 49: 1490, 50: 617, 51: 588, 52: 5875, 53: 867, 54: 251, 55: 2668, 56: 2232, 57: 3969, 58: 120, 59: 1524, 60: 1342, 61: 1213, 62: 1278, 63: 2880, 64: 506, 65: 3899, 66: 77, 67: 694, 68: 1726, 69: 2460, 70: 2327, 71: 582, 72: 905, 73: 306, 74: 1859, 75: 1350, 76: 1857, 77: 57, 78: 1512, 79: 2233, 80: 194, 81: 728, 82: 2440, 83: 3216, 84: 155, 85: 324, 86: 1409, 87: 103, 88: 865, 89: 86, 90: 3795, 91: 1430, 92: 2228, 93: 1082, 94: 4699, 95: 496, 96: 818, 97: 1524, 98: 2824, 99: 335}), (1952976, {0: 197, 1: 387, 2: 728, 3: 101

In [40]:
movie_signatures.cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

PythonRDD[24] at RDD at PythonRDD.scala:53

In [41]:
n_bands = 20
n_rows = 5
similarity_threshold = (1/n_bands)**(1/n_rows)
similarity_threshold

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.5492802716530588

In [42]:
def jaccard_similarity(set_a,set_b):
    intersection = len(set_a.intersection(set_b))
    union = len(set_a.union(set_b))
    return intersection/union

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [43]:
jaccard_similarity({1,2,3},{3,4,5})

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.2

In [44]:
n_lsh_buckets = int(math.sqrt(n_shingles))
lsh_buckets={}


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [45]:
def break_shingles_to_bands(shingles):
    set_list = []
    for _ in range(n_bands):
        set_list.append(set())
        
    for i in shingles:
        set_list[i//n_rows].add(shingles[i])
    return set_list
    

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [46]:
test = {0: 1179,
   1: 138226,
   2: 58526,
   3: 50635,
   4: 63542,
   5: 18022,
   6: 142334,
   7: 90061,
   8: 20921,
   9: 187417,
   10: 66231,
   11: 149743,
   12: 55278,
   13: 16198,
   14: 29504,
   15: 137872,
   16: 26262,
   17: 121676,
   18: 8240,
   19: 55028,
   20: 32945,
   21: 191234,
   22: 61885,
   23: 58198,
   24: 185256,
   25: 21165,
   26: 407631,
   27: 3969,
   28: 26739,
   29: 16283,
   30: 121042,
   31: 21630,
   32: 59449,
   33: 11024,
   34: 91822,
   35: 209568,
   36: 158652,
   37: 87375,
   38: 39936,
   39: 38781,
   40: 31799,
   41: 33956,
   42: 117648,
   43: 121603,
   44: 115801,
   45: 7263,
   46: 25610,
   47: 15775,
   48: 102872,
   49: 99004,
   50: 168497,
   51: 93079,
   52: 192358,
   53: 10220,
   54: 14705,
   55: 45435,
   56: 8949,
   57: 8655,
   58: 58414,
   59: 25409,
   60: 44614,
   61: 143825,
   62: 39789,
   63: 2768,
   64: 164976,
   65: 34187,
   66: 3309,
   67: 1087,
   68: 103689,
   69: 60480,
   70: 129943,
   71: 10567,
   72: 43456,
   73: 6431,
   74: 60255,
   75: 15198,
   76: 5965,
   77: 58259,
   78: 140677,
   79: 36962,
   80: 15882,
   81: 59376,
   82: 146360,
   83: 6505,
   84: 3238,
   85: 64314,
   86: 96034,
   87: 96831,
   88: 138769,
   89: 172868,
   90: 150154,
   91: 115317,
   92: 27755,
   93: 186290,
   94: 6132,
   95: 2436,
   96: 23769,
   97: 84076,
   98: 117144,
   99: 76668}
break_shingles_to_bands(test)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[{50635, 138226, 63542, 1179, 58526}, {18022, 90061, 187417, 20921, 142334}, {29504, 16198, 55278, 149743, 66231}, {121676, 137872, 8240, 55028, 26262}, {191234, 185256, 32945, 58198, 61885}, {3969, 21165, 407631, 26739, 16283}, {91822, 11024, 121042, 59449, 21630}, {209568, 39936, 87375, 158652, 38781}, {121603, 33956, 117648, 31799, 115801}, {25610, 102872, 99004, 7263, 15775}, {192358, 10220, 168497, 14705, 93079}, {25409, 58414, 8655, 8949, 45435}, {44614, 39789, 2768, 143825, 164976}, {60480, 103689, 34187, 3309, 1087}, {43456, 6431, 10567, 129943, 60255}, {36962, 140677, 5965, 58259, 15198}, {3238, 6505, 15882, 59376, 146360}, {96034, 172868, 138769, 64314, 96831}, {150154, 27755, 186290, 6132, 115317}, {2436, 84076, 117144, 23769, 76668}]

In [47]:
def hash_bands(doc_band_set):
    bucket_docs = []
    for s in doc_band_set[1]:
        
        bucket_index = mmh3.hash(str(sorted(s)),31)%n_lsh_buckets
        bucket_docs.append((bucket_index,doc_band_set[0]))
        
    return bucket_docs

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [48]:
hash_band_rdd = movie_signatures.mapValues(break_shingles_to_bands).flatMap(hash_bands)
hash_band_rdd.take(8)
#candidate_proximity_rdd.collect()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[(2291, 595909), (1845, 595909), (539, 595909), (1731, 595909), (2148, 595909), (171, 595909), (421, 595909), (2427, 595909)]

In [54]:
candidate_proximity_rdd = hash_band_rdd.groupByKey()
candidate_proximity_rdd.cache()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

PythonRDD[36] at RDD at PythonRDD.scala:53

In [55]:
#####INTERRUPTED BECAUSE IT NEVER ENDED
candidate_proximity_rdd.take(10)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[(2148, <pyspark.resultiterable.ResultIterable object at 0x7f5660f41fd0>), (698, <pyspark.resultiterable.ResultIterable object at 0x7f5660f41a10>), (496, <pyspark.resultiterable.ResultIterable object at 0x7f5661312090>), (1738, <pyspark.resultiterable.ResultIterable object at 0x7f5660f41b10>), (1374, <pyspark.resultiterable.ResultIterable object at 0x7f5661312290>), (1564, <pyspark.resultiterable.ResultIterable object at 0x7f5661312190>), (1428, <pyspark.resultiterable.ResultIterable object at 0x7f5661312210>), (2246, <pyspark.resultiterable.ResultIterable object at 0x7f5660f41790>), (212, <pyspark.resultiterable.ResultIterable object at 0x7f5661312110>), (1214, <pyspark.resultiterable.ResultIterable object at 0x7f56613120d0>)]

In [56]:
def compare_pairs(it):
    
    
    candidate_list = [x for x in iter(it)]
    pairs = []
    for i in range(len(candidate_list)):
        for j in range(i+1,len(candidate_list)):
            pairs.append((candidate_list[i],candidate_list[j]))
    
    return pairs

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [57]:
candidate_proximity_rdd = candidate_proximity_rdd.mapValues(compare_pairs)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [58]:
def compare_signatures(doc1,doc2):
    #signatures_1 = movie_signatures.lookup(pair[0])
    #signatures_2 = movie_signatures.lookup(pair[1])
    signatures_1 = set(sig_dict.value[doc1].values())
    signatures_2 = set(sig_dict.value[doc2].values())
    #return doc1
    return jaccard_similarity(signatures_1,signatures_2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [59]:
candidate_proximity_rdd.take(1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[(2148, [(595909, 491757), (595909, 19833487), (595909, 317828), (595909, 13298228), (595909, 1237394), (595909, 21499125), (595909, 8678373), (595909, 4520121), (595909, 880056), (595909, 9422274), (595909, 1136182), (595909, 1009516), (595909, 31163724), (595909, 9882012), (595909, 33010153), (595909, 62687), (595909, 5064191), (595909, 11605792), (595909, 33689385), (595909, 14686017), (595909, 5606124), (595909, 22211229), (595909, 20390730), (595909, 15286558), (595909, 9145129), (595909, 11058226), (595909, 10074270), (595909, 2885119), (595909, 13727909), (595909, 2372994), (595909, 950429), (595909, 9976763), (595909, 14737193), (595909, 33471439), (595909, 12224231), (595909, 24486281), (595909, 22376055), (595909, 13677861), (595909, 6173336), (595909, 35589999), (595909, 25584574), (595909, 9835), (595909, 20756695), (595909, 12003939), (595909, 5698193), (595909, 26232501), (595909, 32182348), (595909, 7145079), (595909, 23680613), (595909, 11785619), (595909, 16189500), (5

In [60]:
sig_dict = sc.broadcast(movie_signatures.collectAsMap())

real_pairs = candidate_proximity_rdd.flatMap(lambda candidates: [(c,compare_signatures(*c)) for c in candidates[1]]).filter(lambda a: a[1]>=similarity_threshold and a[1]<0.98).distinct()
real_pairs.take(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[((23430023, 15814364), 0.8348623853211009)]

In [61]:
shingles_dict = sc.broadcast(movie_shingles.collectAsMap())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [62]:
def exact_comparison(docid_1,docid_2):
    set_1 = set(shingles_dict.value[docid_1])
    set_2 = set(shingles_dict.value[docid_2])
    return jaccard_similarity(set_1,set_2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [63]:
#example
exact_comparison(21859037,21858973)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
21859037
Traceback (most recent call last):
  File "<stdin>", line 2, in exact_comparison
KeyError: 21859037



In [64]:
####REMOVE/FILTER DUPLICATES

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [66]:
def given_a_movie(movie_id):
    similar_movies = real_pairs.filter(lambda a: movie_id in list(a[0]) and 0.98>=a[1]>=0.8)
    return similar_movies.collect()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [67]:
given_a_movie(21859037)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[]